In [1]:
import os 
import sys
while os.path.split(os.getcwd())[1] != 'RecSysChallenge2023-Team':
    os.chdir('..')
sys.path.insert(1, os.getcwd())

################################# IMPORT HyperTuning  #################################
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical

################################# IMPORT RECOMMENDERS #################################
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.IR_feature_weighting import okapi_BM_25

################################## IMPORT LIBRARIES ##################################
import scipy.sparse as sps
from tqdm import tqdm
import pandas as pd
import numpy as np
import similaripy
import math 

import Daniele.Utils.MatrixManipulation as mm
import MyDataManager as dm

from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from sklearn.preprocessing import normalize

URMv = dm.getURMviews()
URMo = dm.getURMopen()

urm = mm.defaultExplicitURM(urmv=URMv,urmo=URMo, normalize=False, add_aug=False)
urm.data = np.ones(len(urm.data))

URMv_train, URMv_test = split_train_in_two_percentage_global_sample(urm, train_percentage = 0.80)

urma = mm.augmentURM(URMv_train)
urma.data = np.ones(len(urma.data))
URMfat = sps.vstack([URMv_train,urma])    


recommender = RP3betaRecommender(URMfat)
#{'topK': 106, 'alpha': 0.6155817289031643, 'beta': 0.33427474623540737, 'normalize_similarity': True} -> MAP 0.0276355
recommender.fit(topK= 106, alpha= 0.6155817289031643, beta= 0.33427474623540737, normalize_similarity= True)


/Users/daniele/Desktop/RecSys/RecSysChallenge2023-Team


100%|██████████| 663370/663370 [00:01<00:00, 653234.85it/s]


RP3betaRecommender: URM Detected 3461 (12.4%) items with no interactions.
RP3betaRecommender: Similarity column 27968 (100.0%), 4235.12 column/sec. Elapsed time 6.60 sec


In [2]:
evaluator_test = EvaluatorHoldout(URMv_test, [10])
ris, _ = evaluator_test.evaluateRecommender(recommender)
ris

EvaluatorHoldout: Ignoring 318 ( 0.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 41311 (100.0%) in 12.57 sec. Users per second: 3286


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.013161,0.017691,0.015248,0.005568,0.00758,0.049912,0.018599,0.014128,0.117063,0.052679,...,0.992361,0.116169,0.992361,0.057007,9.526923,0.995026,0.132451,0.709659,3.484374,0.297021
